<a href="https://colab.research.google.com/github/Selma266/Malicious-URL-Detection/blob/main/Malicious_URL_Detection_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Chargement Phishing URLs dataset + prétraitement

In [2]:
import pandas as pd #la bibliothèque Pandas, utilisée pour la manipulation et l’analyse de données tabulaires.
df = pd.read_csv("/content/drive/MyDrive/Phishing URLs_cleaned.csv")  # read dataset url,label  #Chargement du dataset
df.head()

,url,label
0,https://docs.google.com/presentation/d/e/2PACX...,malicious
1,https://btttelecommunniccatiion.weeblysite.com/,malicious
2,https://kq0hgp.webwave.dev/,malicious
3,https://brittishtele1bt-69836.getresponsesite....,malicious
4,https://bt-internet-105056.weeblysite.com/,malicious


Suppression des lignes redondantes

In [3]:
# BEFORE removing duplicates
print("Before removing duplicates:")
print("Shape:", df.shape)

# Step 2: Remove duplicate rows
df = df.drop_duplicates()   #Suppression des doublons

# AFTER removing duplicates
print("\nAfter removing duplicates:")
print("Shape:", df.shape)

Before removing duplicates:
Shape: (504983, 2)

After removing duplicates:
Shape: (504933, 2)


Supprimer les NaN+ Conversion label en valeur Numeriques

In [4]:
# Supprimer les lignes où url ou label est NaN
df = df.dropna(subset=["url", "label"]).reset_index(drop=True)  #reset_index(drop=True) Réinitialise l’index du DataFrame
# strip() supprime les espaces au début et à la fin URL
df["url"] = df["url"].str.strip()
# Conversion des labels texte en valeurs numériques
df["label"] = df["label"].map({"benign": 0, "malicious": 1}).astype(int) #Encodage des labels
print(df["label"].value_counts()) #Vérification de la distribution des classes Afichages

label
0    345738
1    159195
Name: count, dtype: int64


Creation des 3 distributions dataset1(50% 50%),dataset2(90% 10%)et dataset(10%-90%)

In [5]:
#Génère les trois distributions expérimentales à partir du même dataset source.
import pandas as pd
# Sépare le dataset en deux sous-ensembles selon la classe (bénigne / malveillante).
benign = df[df['label'] == 0]
malicious = df[df['label'] == 1]
#Affiche le nombre d’échantillons par classe pour vérifier la disponibilité des données.
print(f"URLs bénignes disponibles: {len(benign)}")
print(f"URLs malveillantes disponibles: {len(malicious)}")
print(f"Total URLs disponibles: {len(df)}")

# Taille fixe pour tous les datasets\ to avoid 'Cannot take a larger sample than population error
TOTAL_SIZE = 176883

# ===== Dataset 1: Équilibré (50-50) =====
def create_balanced_dataset(benign, malicious, total_size=176883):
    """
    Crée un dataset équilibré avec 50% benign et 50% malicious
    """
    size_per_class = total_size // 2  #  176883 // 2 = 88441 pour chaque classe

    ben_sample = benign.sample(n=size_per_class, random_state=42)
    mal_sample = malicious.sample(n=size_per_class, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample]) # Concatène  les deux DataFrames
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 2: 90% bénignes, 10% malveillantes =====
def create_imbalanced_dataset_90_10(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 90% benign et 10% malicious
    """
    ben_size = int(total_size * 0.9)  #90% ≈ 159,194
    mal_size = int(total_size * 0.1)  # 10% ≈ 17,688

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 3: 10% bénignes, 90% malveillantes =====
def create_imbalanced_dataset_10_90(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 10% benign et 90% malicious
    """
    ben_size = int(total_size * 0.1)  # 31,849
    mal_size = int(total_size * 0.9)  # 286,641

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Créer les 3 datasets =====
print("\n" + "="*60)
print("CRÉATION DES DATASETS")
print("="*60)

dataset_balanced = create_balanced_dataset(benign, malicious, TOTAL_SIZE)
dataset_imb_90_10 = create_imbalanced_dataset_90_10(benign, malicious, TOTAL_SIZE)
dataset_imb_10_90 = create_imbalanced_dataset_10_90(benign, malicious, TOTAL_SIZE)

# ===== Afficher les statistiques =====
print("\n📊 DATASET 1 - BALANCED (50-50):")
print(f"   Total samples: {len(dataset_balanced)}")
print(f"   Distribution:")
print(dataset_balanced['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_balanced['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):")
print(f"   Total samples: {len(dataset_imb_90_10)}")
print(f"   Distribution:")
print(dataset_imb_90_10['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_90_10['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):")
print(f"   Total samples: {len(dataset_imb_10_90)}")
print(f"   Distribution:")
print(dataset_imb_10_90['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_10_90['label'].value_counts(normalize=True).sort_index() * 100)


URLs bénignes disponibles: 345738
URLs malveillantes disponibles: 159195
Total URLs disponibles: 504933

CRÉATION DES DATASETS

📊 DATASET 1 - BALANCED (50-50):
   Total samples: 176882
   Distribution:
label
0    88441
1    88441
Name: count, dtype: int64
   Pourcentages:
label
0    50.0
1    50.0
Name: proportion, dtype: float64

📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):
   Total samples: 176882
   Distribution:
label
0    159194
1     17688
Name: count, dtype: int64
   Pourcentages:
label
0    90.000113
1     9.999887
Name: proportion, dtype: float64

📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):
   Total samples: 176882
   Distribution:
label
0     17688
1    159194
Name: count, dtype: int64
   Pourcentages:
label
0     9.999887
1    90.000113
Name: proportion, dtype: float64
